In [7]:
%load_ext autoreload
%autoreload 2

In [40]:
import load
import model
import numpy as np

In [29]:
train_path = "FSC_P4_Streams\Audio\Streams\Train"
train_labels = "FSC_P4_Streams\Transcripts\SAD\Train"
dev_path = "FSC_P4_Streams\Audio\Streams\Dev"
dev_labels = "FSC_P4_Streams\Transcripts\SAD\Dev"


data_loader = load.LoadAudio(debug=False)
X, audio_info_list, Y = data_loader.load_all(train_path, train_labels)

Loading audio from FSC_P4_Streams\Audio\Streams\Train
Loaded 2 audio files
Loading labels from FSC_P4_Streams\Transcripts\SAD\Train
Loaded 2 labels


In [38]:
import torch
from torch.utils.data import Dataset, DataLoader

In [45]:
class SADDataset(Dataset):
    def __init__(self, X, Y, min_len=None):
        self.X = X  # List of feature matrices
        self.Y = Y  # List of labels (0/1)
        self.min_len = min_len or min([len(x) for x in X])

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        x = torch.tensor(self.X[idx], dtype=torch.float32)
        y = torch.tensor(self.Y[idx], dtype=torch.float32)
        # Pad the sequences to max_len
        # x_padded = F.pad(x, (0, 0, 0, self.max_len - len(x)))  # Pads time axis
        x_cut = x[:self.min_len]
        y_cut = y[:self.min_len]
        return x_cut, y_cut


dataset = SADDataset(X, Y)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

input_size = 40  # Example: feature size from MFCC or spectrogram
hidden_size = 256  # This can be scaled up for larger models
num_layers = 3  # Number of LSTM layers

sad_model = model.SADModel(input_size, hidden_size, num_layers)

print("training model")
for batch_x, batch_y in dataloader:
    output = sad_model(batch_x)  # Forward pass
print("finished training model")
print("last batch: ", output.shape, "mean: ", output.mean())


    


tensor([[[0.4755],
         [0.4782],
         [0.4799],
         ...,
         [0.4643],
         [0.4643],
         [0.4643]]], grad_fn=<SigmoidBackward0>)
tensor([[[0.4712],
         [0.4701],
         [0.4695],
         ...,
         [0.4649],
         [0.4670],
         [0.4694]]], grad_fn=<SigmoidBackward0>)


In [58]:
print(output.shape)  # (batch_size, seq_len, 1)
print(output[0, 0, 0])
print(torch.mean(output)) 

torch.Size([1, 180001, 1])
tensor(0.4712, grad_fn=<SelectBackward0>)
tensor(0.4673, grad_fn=<MeanBackward0>)
